## Demostration of the ```add_ontology_subtree``` endpoint

We demonstrate the ability to add any ontology term to the MIRA DKG given its curie. We also provide the option to add all taxonomy terms in the subtree of the specified taxonomy term to add. Currently we only support the addition of **ncbitaxonomy** terms. This demonstration will make use of the ncbitaxonomy term here: https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Undef&id=9871&lvl=3&lin=f&keep=1&srchmode=1&unlock. 

In [97]:
import requests
from mira.dkg.client import Neo4jClient

## We define an instance of the Neo4jClient to query for the nodes and edges added to the DKG

In [98]:
client = Neo4jClient()

## Define the ncbitaxonomy curie to add.
We first only add term itself and no terms under its subtree to the DKG.

In [99]:
curie = 'ncbitaxon:9871'
add_subtree = False
params = {"curie": curie, "add_subtree": False}

In [102]:
response = requests.post("http://localhost:8000/api/add_ontology_subtree", params=params)
response.status_code

200

## Query for the added node

We also query for any relations where the added node is the target of the relation under relation type ```subclass_of```. Children terminology have a directed ```subclass_of``` relation towards the parent term in the ncbitaxonomy. The absence of any relation information where ```ncbitaxon:9871``` is the target node under relation type ```subclass_of``` shows that we only added the node.

In [106]:
# The ncbitaxonomy term with identifier 9871 has name "Odocoileus" in the DKG which corresponds to this term: 
# https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Info&id=9871&lvl=3&lin=f&keep=1&srchmode=1&unlock

query = f"MATCH (N) WHERE N.id = 'ncbitaxon:9871' RETURN N"
print(client.query_tx(query)[0][0]._properties)

{'property_values': ['NCBITaxon:genus'], 'xref_types': [''], 'property_predicates': ['has_rank'], 'obsolete': False, 'xrefs': ['gc_id:1'], 'name': 'Odocoileus', 'id': 'ncbitaxon:9871', 'type': 'class'}


In [104]:
relation_query = f"MATCH (source)-[r:subclass_of]->(target) WHERE target.id = 'ncbitaxon:9871' RETURN source, r, target"
result = client.query_tx(relation_query)
print(result)

[]


## Add the ontology subtree full of terms under the term

In [105]:
params = {"curie": curie, "add_subtree": True}
response = requests.post("http://localhost:8000/api/add_ontology_subtree", params=params)
response.status_code

200

## Relationship info has been added
We now see that there is a plethora of relationship info added where ```ncbitaxon:9871``` is the target node now under relation type ```subclass_of```. The number of relations added under this constraint is 5 because there are 5 direct ncbitaxonomy terms under ```ncbitaxon:9871```. 

In [119]:
relation_query = f"MATCH (source)-[r:subclass_of]->(target) WHERE target.id = 'ncbitaxon:9871' RETURN source, r, target"
result = client.query_tx(relation_query)
print(f"The number of relations added to the EPI DKG where the target node has curie 'ncbitaxon:9871' and the relation type is 'subclass_of' is {len(result)}")

The number of relations added to the EPI DKG where the target node has curie 'ncbitaxon:9871' and the relation type is 'subclass_of' is 5


In [137]:
print(f"Source Node: {result[0][0]}") 
print()
print(f"Relation Type: {result[0][1].type}, {result[0][1].properties}")
print()
print(f"Target Node: {result[0][2]}")

Source Node: <Node element_id='3099' labels=frozenset() properties={'xref_types': [''], 'xrefs': ['gc_id:1'], 'name': 'unclassified Odocoileus', 'obsolete': False, 'id': 'ncbitaxon:2625720', 'type': 'class'}>



AttributeError: 'subclass_of' object has no attribute 'properties'

In [146]:
result[0][1].properties

AttributeError: 'subclass_of' object has no attribute 'properties'